##### 展示数据填充率，标准差等信息

In [ ]:
pd.set_option('display.max_columns',200)
train_B.describe()

In [ ]:
#查看数据类型
train_B.info()
train_B.dtypes[train_B.dtypes == np.int64]


In [ ]:
#查看数据内容类别
train_B['UserInfo_170'].unique()

train_B = train_B.drop('UserInfo_170',axis=1)


- 严重缺失的特征会给模型带来极大的噪音,模型在学习的过程中会被较大程度的干扰,所以为了增加模型的鲁棒性,我们考虑将含有大的噪音的数据进行删除.此处我们设立阈值为1%,我们将缺失的特征大于99%的特征删除.(阈值可以自己进行调整).

In [ ]:
train_B_info = train_B.describe()

meaningful_col = []
for col in train_B_info.columns: 
    if train_B_info.ix[0,col] > train_B.shape[0] * 0.01:
           meaningful_col.append(col)

- 关于缺失值的处理,不同的问题有不同的处理方式,有的时候我们需要对其进行删除,有时会进行填充(填充的方式也有不同,后续我们会选择特定的问题进行阐述),此处我们仅仅介绍和贷款类业务相关的处理:

- 在贷款业务中,很多缺失值是有实际意义的,它们往往来源于用户故意不填写,或者没有,所以我们没法搜集这些数据,这个需要一些简单的业务理解.而针对这样的情况,一个非常有效的方式是直接填充某一个值(可以是-100,999等其他值)[6],此处我们直接用-999进行填充.

#### 高线性相关性数据处理

- 如果两个特征是完全线性相关的,这个时候我们仅仅只需要保留其中一个即可.因为第二个特征包含的信息基本完全被第一个特征所包含.此时如果两个特征同时保留的话,模型的性能很大情况下会出现下降的情况.

In [ ]:
# 我们选择将高线性相关的特征进行删除[7].
relation = train_B_1.corr()
length = relation.shape[0]
high_corr = list()
final_cols = []
del_cols =[]
for i in range(length):
    if relation.columns[i] not in del_cols:
        final_cols.append(relation.columns[i])
        for j in range(i+1,length):
            if (relation.iloc[i,j] > 0.98) and (relation.columns[j] not in del_cols):
                del_cols.append(relation.columns[j])

train_B_1 = train_B_1[final_cols]


- 为了能够复现成绩,注意在每个算法的包中都有一个随机种子,在XGBoost里面就是random_seed

In [ ]:
import xgboost as xgb

Trate=0.25  
params = {'booster':'gbtree',
                  'eta': 0.1,
                   'max_depth': 4,
                   'max_delta_step': 0,
                   'subsample':0.9,      
                   'colsample_bytree': 0.9,
                   'base_score': Trate,
                   'objective': 'binary:logistic',
                   'lambda':5,
                   'alpha':8,
                   'random_seed':100
                   }
params['eval_metric'] = 'auc' 
xgb_model = xgb.train(params,dtrain_B,num_boost_round=200,maximize=True,verbose_eval=True)  
